In [3]:
# Important libraries
from fastai.vision.all import *
from pathlib import Path
from time import sleep
from fastdownload import download_url
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    with DDGS() as ddgs:
        # generator which yields dicts with:
        # {'title','image','thumbnail','url','height','width','source'}
        search_results = ddgs.images(keywords=term)       
        # grap number of max_images urls
        image_urls = [next(iter(search_results)).get("image") for _ in range(max_images)]
        # convert to L (functionally extended list class from fastai)
        return L(image_urls)

#search categories
searches = ('forest', 'bird')

# Main folder
path = Path('bird_or_not')


# Search loop
for category in searches:
  # Forming sub-folder based on categories.
  dest = path / category
  dest.mkdir(exist_ok=True, parents=True)
  print(f"Downloading images for category: {category}")
  download_images(dest, urls=search_images(f'{category} photo'))
  sleep(10)# Pause between searches to avoid over-loading server
  download_images(dest, urls=search_images(f'{category} sun photo'))
  sleep(10)
  download_images(dest, urls=search_images(f'{category} shade photo'))
  sleep(10)
  resize_images(path/category, max_size=400, dest=path/category)

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)#remove failed images

# Prepare data for training
dls = DataBlock(
  blocks=(ImageBlock, CategoryBlock),
  get_items=get_image_files, 
  splitter=RandomSplitter(valid_pct=0.2, seed=42),
  get_y=parent_label,  
  item_tfms=[Resize(192, method='squish')] 
).dataloaders(path, bs=32)

# Create and fine-tune the image classification model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)  

# Make a prediction on a new image
urls = search_images('forest photos', max_images=1)
dest = 'forest.jpg'
download_url(urls[0], dest, show_progress=False)
im = Image.open(dest)
im.to_thumb(256,256)
is_bird, _, probs = learn.predict(PILImage.create("forest.jpg"))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")


Searching for 'forest photo'
Searching for 'forest sun photo'
Searching for 'forest shade photo'
Searching for 'bird photo'
Searching for 'bird sun photo'
Searching for 'bird shade photo'


epoch,train_loss,valid_loss,error_rate,time
0,0.292947,0.000000,0.000000,00:56


epoch,train_loss,valid_loss,error_rate,time
0,0.000008,0.000000,0.000000,01:08
1,0.000062,0.000014,0.000000,01:07
2,0.000063,0.000000,0.000000,00:58


Searching for 'forest photos'


This is a: forest.
Probability it's a bird: 0.0000
